# WaterLLLMarks Pipeline

In [1]:
%load_ext autoreload
%autoreload 2

import nest_asyncio

nest_asyncio.apply()

## LLM Papers dataset

Paper got from AutoRAG's evaluation pipeline. The dataset is composed of text chunks automatically extracted from the papers' PDFs using OCR. This makes it relatively low quality, as there is a lot of noise.



In [2]:
from datasets import load_dataset

corpus = load_dataset("MarkrAI/AutoRAG-evaluation-2024-LLM-paper-v1", "corpus")
qa = load_dataset("MarkrAI/AutoRAG-evaluation-2024-LLM-paper-v1", "qa")

In [3]:
from langchain_openai import ChatOpenAI

client = ChatOpenAI(model="gpt-3.5-turbo")


In [4]:
from langchain_chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings


docs = corpus["train"]["contents"]
ids = corpus["train"]["doc_id"]

embedding_fn = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_client = Chroma(embedding_function=embedding_fn)
for i in range(0, len(docs), 1000):
    chroma_client.add_texts(texts=docs[i : i + 1000], ids=ids[i : i + 1000])

retriever = chroma_client.as_retriever(search_kwargs={"k": 3})


In [5]:
from waterllmarks.pipeline import ExecConfig, RAGLLM, VectorDB, Evaluate

pipeline = VectorDB() | RAGLLM() | Evaluate()
pipeline.configure(ExecConfig(vectordb=retriever, llm=client, embedding=embedding_fn))

qaset = [
    {
        "query": entry["query"],
        "reference": entry["generation_gt"][0],
    }
    for entry in qa["train"]  # .select(range(10))
]
res = pipeline.apply(qaset[20:50], max_workers=10)
res

  7%|▋         | 2/30 [00:25<05:59, 12.83s/it]
  + Exception Group Traceback (most recent call last):
  |   File "/home/leo/Codespace/leolavaur.git/llm-text-watermarking/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_165361/2906848695.py", line 13, in <module>
  |     res = pipeline.apply(qaset[20:50], max_workers=10)
  |           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/leo/Codespace/leolavaur.git/llm-text-watermarking/waterllmarks/pipeline.py", line 173, in apply
  |     return [f.result() for f in iterator]
  |             ^^^^^^^^^^
  |   File "/home/leo/.local/share/uv/python/cpython-3.12.6-linux-x86_64-gnu/lib/python3.12/concurrent/futures/_base.py", line 449, in result
  |     return self.__get_result()
  |            ^^^^^^^^^^^^^^^^^^^
  |   File "/home/leo/.local/share/uv/python/cpython-3.12.6-linux-x86_64-gnu/lib/python3.12

In [ ]:
from waterllmarks.pipeline import LLM

test_query = {
    "query": "Does the Turing Test assess a machine's ability to exhibit intelligent behavior equivalent to that of a human?",
    "reference": "Yes",
    "chunks": [
        "# (Ir)Rationality And Cognitive Biases In Large Language Models\n## Rationality Task Results\n\nResults across all tasks are aggregated in Table 3 and Figure 3. The model that displayed the best overall performance was OpenAI's GPT-4, which achieved the highest proportion of answers that were correct and where the results was achieved through correct reasoning (cateogorised as correct and *human-like* in the above categorisation). GPT-4 gave the correct response and correct reasoning in 69.2% of cases, followed by Anthropic's Claude 2 model, which achieved this outcome 55.0% of the time. Conversely, the model with the highest proportion of incorrect responses (both human-like and non-human-like) was Meta's Llama 2 model with 7 billion parameters, which gave incorrect responses in 77.5% of cases. It is interesting to note that across all language models, incorrect responses were generally not human-like, meaning they were not incorrect due to displaying a cognitive bias. Instead, these responses generally displayed illogical reasoning, and even on occasion provided correct reasoning but then gave an incorrect final answer. An example of the latter is illustrated in Figure 4: this example shows Bard's response to the facilitated version of the Wason task, where the correct response is that both Letter 3 and Letter 4 should be turned over. The model correctly reaches this conclusion in the explanation, but both at the start and end of the response only states that Letter 4 needs to be turned over. This type of response, where the reasoning is correct but the final answer is not, was observed across all model families to varying degrees.  \nThe result that most incorrect responses were not incorrect due to having fallen for a cognitive bias highlight that these models do not fail at these tasks in the same way that humans do. As we have seen, many studies have shown that LLMs simulate human biases and societal norms [24–26]. However, when it comes to reasoning, the effect is less clear. The model that displayed the highest proportion of human-like biases in its responses was GPT-3.5, where this only occurred in 21.7% of cases. If we include human-like correct responses for GPT-3.5, this brings the proportion to 50.8% of cases. Again, the model that displayed the most human-like responses (both correct and incorrect) was GPT-4 (73.3%); the lowest was Llama 2 with 13 billion parameters, only giving human-like responses in 8.3% of cases. The comparison between correct and",
        "# (Ir)Rationality And Cognitive Biases In Large Language Models\n## 1 Introduction\n\n can be used to test the rationality of a model; this is a complex problem which requires a consensus what is deemed rational and irrational.  \nMirco Musolesi University College London University of Bologna m.musolesi@ucl.ac.uk In using methods designed to evaluate human reasoning, it is important to acknowledge the performance vs. competence debate [14]. This line of argument encourages *species-fair* comparisons between humans and machines, meaning that we should design tests specific to either humans or machines, as otherwise apparent failures may not reflect underlying capabilities but only superficial differences. Lampinen[15] discusses this problem when it comes to language models in particular, highlighting that different approaches must be taking to evaluate cognitive and foundation models. However, if we take the purpose of LLMs to be to produce human-like language, perhaps the best approach is precisely to evaluate their output with tasks designed to evaluate humans. This is the approach we have taken in this paper - in order to identify whether LLMs reason rationally, or whether they exhibit biases that can be assimilated to those present in human decision-making, the most appropriate approach is therefore to use tasks that were initially designed for humans. Building on this debate and looking at LLMs being evaluated using human tests, Hagendorff et al. [16] have proposed the creation of a new field of research called *machine psychology*, which would treat LLMs as participants in psychological experiments. The approach employed in this paper precisely applies tests from psychology that were originally designed for humans, in this case to evaluate rational and irrational reasoning displayed but such models. Further to this, some have even discussed the potential of using LLMs as participants in cognitive experiments instead of humans [17], although some see this proposal as too optimistic [18], and others warn against excessive anthropomorphism [19]. One argument against the use of such models in cognitive experiments is that LLMs may be effective at approximating average human judgements, but are not good at capturing the variation in human behaviour [20]. One potential avenue to address this issue is current work on language models impersonating different roles [21], in this way capturing some of the variation in human behaviour. Binz and Schulz [22] show that after finetuning LLMs on data from psychological experiments, they can become accurate cognitive models, which they claim begins paving the way for the potential of using these models to study human behaviour. Park et al. [23] combine large language models with computational interactive agents to simulate human behaviour",
        '## 1. Introduction\n\nIn the recent years, Large Language Models (LLMs) have been among the most impressive achievements in Artificial intelligence (AI) and Machine Learning (ML), raising questions about whether we are close to an Artificial General Intelligence (AGI) system (Bubeck et al., 2023; Fei et al.,\n2022). Foundation models like ChatGPT (OpenAI, 2023b) have shown remarkable performance across a wide range of novel and complex tasks, including coding, vision, healthcare, law, education, and psychology (Bubeck et al., 2023).  \nThese models perform close to human experts, without needing additional re-training or fine-tuning, apparently mimicking human abilities and reasoning (Guo et al., 2023; Min et al., 2021; Bubeck et al., 2023). In particular, recent work has demonstrated how LLMs can emulate human reasoning in solving complex problems: LLMs can simulate chain of thoughts, generating a reasoning path to decompose complex problems into multiple easier steps to solve (Kojima et al., 2022; Wei et al., 2022b; Ho et al., 2022). We could conclude that LLMs are able to replicate high-level cognitive and reasoning capabilities of humans (Huang & Chang, 2022; Hagendorff et al., 2022), especially with bigger foundation models like GPT-4 (Wei et al., 2022a). If we assume this is true, a natural question arises: "Can we leverage LLMs to simulate aspects of human behavior that deviate from perfect rationality so as to eventually improve our understanding of diverse human conduct?"\nModeling human behavior has seen significant interest in various domains, ranging from robotics for human-robot collaboration (Dragan et al., 2015), to finance and economics for modeling human investors and consumers (Liu et al., 2022; Akerlof & Shiller, 2010). In studies of human-robot interactions, it is commonly embraced that humans must not be modeled as optimal agents (Carroll et al., 2019), and an irrational human when correctly modeled, can communicate more information about their objectives than a perfectly rational human can (Chan et al., 2021). For instance, humans exhibit bounded rationality (Simon, 1997), often making satisfactory but not optimal decisions due to their limited knowledge and processing power. They are also known to be myopic in their decision making as they care more about short-term rewards than long-term rewards. While the discount factor in reinforcement learning (RL)',
        "# (Ir)Rationality And Cognitive Biases In Large Language Models\n## 1 Introduction\n\n we use tasks from the cognitive psychology literature designed to test human cognitive biases, and apply these to a series of LLMs to evaluate whether they display rational or irrational reasoning. The capabilities of these models are quickly advancing, therefore the aim of this paper is to provide a methodological contribution showing how we can assess and compare LLMs. A number of studies have taken a similar approach, however they do not generally compare across different model types [12, 16, 29–35], or those that do are not evaluating rational reasoning [36]. Some find that LLMs outperform humans on reasoning tasks [16, 37], others find that these models replicate human biases [30, 38], and finally some studies have shown that LLMs perform much worse than humans on certain tasks [36]. Binz and Schulz [12] take a similar approach to that presented in this paper, where they treat GPT-3 as a participant in a psychological experiment to assess its decision-making, information search, deliberation and causal reasoning abilities. They assess the responses across two dimensions, looking at whether GPT-3's output is correct and/or human-like; we follow this approach in this paper as it allows us to distinguish between answers that are incorrect due to a human-like bias or are incorrect in a different way. While they find that GPT-3 performs as well or even better than human subjects, they also find that small changes to the wording of tasks can dramatically decrease the performance, likely due to GPT-3 having encountered these tasks in training. Hagendorff et al. [16] similarly use the Cognitive Reflection Test (CRT) and semantic illusions on a series of OpenAI's Generative Pre-trained Transformer (GPT) models. They classify the responses as *correct, intuitive* (but incorrect), and *atypical* - as models increase in size, the majority of responses go from being atypical, to intuitive, to overwhelmingly correct for GPT-4, which no longer displays human cognitive errors. Other studies that find the reasoning of LLMs to outperform that of humans includes Chen et al.'s [33] assessment of the economic rationality of GPT, and Webb et al.'s [34] comparison of GPT-3 and human performance on analogical tasks.  \nAs mentioned, some studies have found that LLMs replicate cognitive biases present in human reasoning, and so in some instances display irrational thinking in the same way that humans do. Itzhak et al",
        "# How Reliable Are Automatic Evaluation Methods For Instruction-Tuned Llms?\n## 4.1 Human Evaluation\n\n        |\n| ENSV_SV    |           |           |           |\n| 35         |           |           |           |\n| (36)       |           |           |           |\n| 68         |           |           |           |\n| (63)       |           |           |           |\n| 99         |           |           |           |\n| (85)       |           |           |           |\n| Avg. diff. |           |           |           |\n| 1          | .         |         3 |         4 |  \ncorrectness since it is the most important criterion for instruction tuning and inherently encompasses relatedness. Moreover, the higher degree of alignment between GPT-4 and humans on correctness compared to the other criteria prompts the need for a deeper analysis.",
    ],
    "chunk_ids": [
        "3e92c39c-6707-4cf9-aca5-c6c84f72309b",
        "a09c2e9a-dd24-4e36-9b2e-7d4e15e58639",
        "ed69de92-b214-4de5-ac6a-6c2e204e6416",
        "a7145295-1155-4d88-baaf-48ec36095ae4",
        "9122b070-3595-4f52-9367-f26553930c70",
    ],
    "context": "# (Ir)Rationality And Cognitive Biases In Large Language Models\n## Rationality Task Results\n\nResults across all tasks are aggregated in Table 3 and Figure 3. The model that displayed the best overall performance was OpenAI's GPT-4, which achieved the highest proportion of answers that were correct and where the results was achieved through correct reasoning (cateogorised as correct and *human-like* in the above categorisation). GPT-4 gave the correct response and correct reasoning in 69.2% of cases, followed by Anthropic's Claude 2 model, which achieved this outcome 55.0% of the time. Conversely, the model with the highest proportion of incorrect responses (both human-like and non-human-like) was Meta's Llama 2 model with 7 billion parameters, which gave incorrect responses in 77.5% of cases. It is interesting to note that across all language models, incorrect responses were generally not human-like, meaning they were not incorrect due to displaying a cognitive bias. Instead, these responses generally displayed illogical reasoning, and even on occasion provided correct reasoning but then gave an incorrect final answer. An example of the latter is illustrated in Figure 4: this example shows Bard's response to the facilitated version of the Wason task, where the correct response is that both Letter 3 and Letter 4 should be turned over. The model correctly reaches this conclusion in the explanation, but both at the start and end of the response only states that Letter 4 needs to be turned over. This type of response, where the reasoning is correct but the final answer is not, was observed across all model families to varying degrees.  \nThe result that most incorrect responses were not incorrect due to having fallen for a cognitive bias highlight that these models do not fail at these tasks in the same way that humans do. As we have seen, many studies have shown that LLMs simulate human biases and societal norms [24–26]. However, when it comes to reasoning, the effect is less clear. The model that displayed the highest proportion of human-like biases in its responses was GPT-3.5, where this only occurred in 21.7% of cases. If we include human-like correct responses for GPT-3.5, this brings the proportion to 50.8% of cases. Again, the model that displayed the most human-like responses (both correct and incorrect) was GPT-4 (73.3%); the lowest was Llama 2 with 13 billion parameters, only giving human-like responses in 8.3% of cases. The comparison between correct and\n\n# (Ir)Rationality And Cognitive Biases In Large Language Models\n## 1 Introduction\n\n can be used to test the rationality of a model; this is a complex problem which requires a consensus what is deemed rational and irrational.  \nMirco Musolesi University College London University of Bologna m.musolesi@ucl.ac.uk In using methods designed to evaluate human reasoning, it is important to acknowledge the performance vs. competence debate [14]. This line of argument encourages *species-fair* comparisons between humans and machines, meaning that we should design tests specific to either humans or machines, as otherwise apparent failures may not reflect underlying capabilities but only superficial differences. Lampinen[15] discusses this problem when it comes to language models in particular, highlighting that different approaches must be taking to evaluate cognitive and foundation models. However, if we take the purpose of LLMs to be to produce human-like language, perhaps the best approach is precisely to evaluate their output with tasks designed to evaluate humans. This is the approach we have taken in this paper - in order to identify whether LLMs reason rationally, or whether they exhibit biases that can be assimilated to those present in human decision-making, the most appropriate approach is therefore to use tasks that were initially designed for humans. Building on this debate and looking at LLMs being evaluated using human tests, Hagendorff et al. [16] have proposed the creation of a new field of research called *machine psychology*, which would treat LLMs as participants in psychological experiments. The approach employed in this paper precisely applies tests from psychology that were originally designed for humans, in this case to evaluate rational and irrational reasoning displayed but such models. Further to this, some have even discussed the potential of using LLMs as participants in cognitive experiments instead of humans [17], although some see this proposal as too optimistic [18], and others warn against excessive anthropomorphism [19]. One argument against the use of such models in cognitive experiments is that LLMs may be effective at approximating average human judgements, but are not good at capturing the variation in human behaviour [20]. One potential avenue to address this issue is current work on language models impersonating different roles [21], in this way capturing some of the variation in human behaviour. Binz and Schulz [22] show that after finetuning LLMs on data from psychological experiments, they can become accurate cognitive models, which they claim begins paving the way for the potential of using these models to study human behaviour. Park et al. [23] combine large language models with computational interactive agents to simulate human behaviour\n\n## 1. Introduction\n\nIn the recent years, Large Language Models (LLMs) have been among the most impressive achievements in Artificial intelligence (AI) and Machine Learning (ML), raising questions about whether we are close to an Artificial General Intelligence (AGI) system (Bubeck et al., 2023; Fei et al.,\n2022). Foundation models like ChatGPT (OpenAI, 2023b) have shown remarkable performance across a wide range of novel and complex tasks, including coding, vision, healthcare, law, education, and psychology (Bubeck et al., 2023).  \nThese models perform close to human experts, without needing additional re-training or fine-tuning, apparently mimicking human abilities and reasoning (Guo et al., 2023; Min et al., 2021; Bubeck et al., 2023). In particular, recent work has demonstrated how LLMs can emulate human reasoning in solving complex problems: LLMs can simulate chain of thoughts, generating a reasoning path to decompose complex problems into multiple easier steps to solve (Kojima et al., 2022; Wei et al., 2022b; Ho et al., 2022). We could conclude that LLMs are able to replicate high-level cognitive and reasoning capabilities of humans (Huang & Chang, 2022; Hagendorff et al., 2022), especially with bigger foundation models like GPT-4 (Wei et al., 2022a). If we assume this is true, a natural question arises: \"Can we leverage LLMs to simulate aspects of human behavior that deviate from perfect rationality so as to eventually improve our understanding of diverse human conduct?\"\nModeling human behavior has seen significant interest in various domains, ranging from robotics for human-robot collaboration (Dragan et al., 2015), to finance and economics for modeling human investors and consumers (Liu et al., 2022; Akerlof & Shiller, 2010). In studies of human-robot interactions, it is commonly embraced that humans must not be modeled as optimal agents (Carroll et al., 2019), and an irrational human when correctly modeled, can communicate more information about their objectives than a perfectly rational human can (Chan et al., 2021). For instance, humans exhibit bounded rationality (Simon, 1997), often making satisfactory but not optimal decisions due to their limited knowledge and processing power. They are also known to be myopic in their decision making as they care more about short-term rewards than long-term rewards. While the discount factor in reinforcement learning (RL)\n\n# (Ir)Rationality And Cognitive Biases In Large Language Models\n## 1 Introduction\n\n we use tasks from the cognitive psychology literature designed to test human cognitive biases, and apply these to a series of LLMs to evaluate whether they display rational or irrational reasoning. The capabilities of these models are quickly advancing, therefore the aim of this paper is to provide a methodological contribution showing how we can assess and compare LLMs. A number of studies have taken a similar approach, however they do not generally compare across different model types [12, 16, 29–35], or those that do are not evaluating rational reasoning [36]. Some find that LLMs outperform humans on reasoning tasks [16, 37], others find that these models replicate human biases [30, 38], and finally some studies have shown that LLMs perform much worse than humans on certain tasks [36]. Binz and Schulz [12] take a similar approach to that presented in this paper, where they treat GPT-3 as a participant in a psychological experiment to assess its decision-making, information search, deliberation and causal reasoning abilities. They assess the responses across two dimensions, looking at whether GPT-3's output is correct and/or human-like; we follow this approach in this paper as it allows us to distinguish between answers that are incorrect due to a human-like bias or are incorrect in a different way. While they find that GPT-3 performs as well or even better than human subjects, they also find that small changes to the wording of tasks can dramatically decrease the performance, likely due to GPT-3 having encountered these tasks in training. Hagendorff et al. [16] similarly use the Cognitive Reflection Test (CRT) and semantic illusions on a series of OpenAI's Generative Pre-trained Transformer (GPT) models. They classify the responses as *correct, intuitive* (but incorrect), and *atypical* - as models increase in size, the majority of responses go from being atypical, to intuitive, to overwhelmingly correct for GPT-4, which no longer displays human cognitive errors. Other studies that find the reasoning of LLMs to outperform that of humans includes Chen et al.'s [33] assessment of the economic rationality of GPT, and Webb et al.'s [34] comparison of GPT-3 and human performance on analogical tasks.  \nAs mentioned, some studies have found that LLMs replicate cognitive biases present in human reasoning, and so in some instances display irrational thinking in the same way that humans do. Itzhak et al\n\n# How Reliable Are Automatic Evaluation Methods For Instruction-Tuned Llms?\n## 4.1 Human Evaluation\n\n        |\n| ENSV_SV    |           |           |           |\n| 35         |           |           |           |\n| (36)       |           |           |           |\n| 68         |           |           |           |\n| (63)       |           |           |           |\n| 99         |           |           |           |\n| (85)       |           |           |           |\n| Avg. diff. |           |           |           |\n| 1          | .         |         3 |         4 |  \ncorrectness since it is the most important criterion for instruction tuning and inherently encompasses relatedness. Moreover, the higher degree of alignment between GPT-4 and humans on correctness compared to the other criteria prompts the need for a deeper analysis.",
    "answer": "Yes, the Turing Test assesses a machine's ability to exhibit intelligent behavior equivalent to that of a human.",
}

llm = LLM(config=pipeline.config)
llm(test_query)

{'query': "Does the Turing Test assess a machine's ability to exhibit intelligent behavior equivalent to that of a human?",
 'reference': 'Yes',
 'chunks': ["# (Ir)Rationality And Cognitive Biases In Large Language Models\n## Rationality Task Results\n\nResults across all tasks are aggregated in Table 3 and Figure 3. The model that displayed the best overall performance was OpenAI's GPT-4, which achieved the highest proportion of answers that were correct and where the results was achieved through correct reasoning (cateogorised as correct and *human-like* in the above categorisation). GPT-4 gave the correct response and correct reasoning in 69.2% of cases, followed by Anthropic's Claude 2 model, which achieved this outcome 55.0% of the time. Conversely, the model with the highest proportion of incorrect responses (both human-like and non-human-like) was Meta's Llama 2 model with 7 billion parameters, which gave incorrect responses in 77.5% of cases. It is interesting to note that acro

## RAG-12000 dataset

In [6]:
from datasets import load_dataset

ds = load_dataset("neural-bridge/rag-dataset-12000")


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

ValueError: BuilderConfig 'corpus' not found. Available: ['default']